In [1]:
import pandas as pd
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
import os
import chromedriver_autoinstaller
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options

from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support.ui import Select
chrome_options = Options()
chrome_options.add_argument("--start-maximized")
chromedriver_autoinstaller.install()

'/home/bilal/.local/lib/python3.8/site-packages/chromedriver_autoinstaller/106/chromedriver'

In [2]:

scrape_link="https://toplaxrecruits.com/"

In [3]:
output_folder_name="../testing/toplaxrecruits"
if (not os.path.exists(output_folder_name)):
    os.mkdir(output_folder_name)

In [4]:
driver = webdriver.Chrome(options=chrome_options)

In [5]:
print(f"Getting {scrape_link}")
driver.get(scrape_link)
sleep(2)
menu_items=["menu-item-105016","menu-item-105007"]

Getting https://toplaxrecruits.com/


In [6]:
categories={}

print("getting all the sub menus for girls and boys commitments")
for menu in menu_items:
    
    comitments = driver.find_element(By.XPATH,f'.//li[@id="{menu}"]')
    # Create an ActionChains object
    actions = ActionChains(driver)

    # Hover over the element
    actions.move_to_element(comitments).perform()
    sleep(1)
    sub_list=comitments.find_elements(By.XPATH,".//li")
    len(sub_list)
    for sub_menu in sub_list:
        print(sub_menu.text)
        href_value = sub_menu.find_element(By.XPATH,".//a").get_attribute("href")
        print(href_value)

        categories[sub_menu.text]=href_value

getting all the sub menus for girls and boys commitments
2025 Boys
https://toplaxrecruits.com/2025-boys
2024 Boys
https://toplaxrecruits.com/2024-boys
2023 Boys
https://toplaxrecruits.com/2023-boys-commits
2025 Girls
https://toplaxrecruits.com/2025-girls
2024 Girls
https://toplaxrecruits.com/2024-girls
2023 Girls
https://toplaxrecruits.com/2023-girls


In [7]:
for cat in categories.keys():
# cat="2024 Boys"
    print(f"scrapping data for {cat}\n")
    driver.get(categories[cat])
    sleep(2)
    print("show 100 records per page\n")
    select = Select(driver.find_element(By.XPATH,'.//select'))

    select.select_by_value("100")
    sleep(2)
    dfs_list=[]
    print("scrolling through all the pages....\n")
    while True:
        sleep(3)

        driver.find_element(By.XPATH,'.//table')

        results=driver.find_elements(By.XPATH,'.//table')
        table_html = results[-1].get_attribute("outerHTML")

        # Use Pandas to read HTML and convert it to a DataFrame
        df = pd.read_html(table_html)[0]
        df = df.dropna(axis = 0, how = 'all')
        if df.shape[0] == 0:
            break

        try:
            df.drop(['Unnamed: 8'], axis = 1, inplace = True) 
        except:
            pass

        dfs_list.append(df)
        try:
            driver.find_element(By.XPATH,'.//a[@class="paginate_button next"]').click()

        except:
            break


    result_df=pd.concat(dfs_list, axis=0, ignore_index=True)
    result_df
    print(f"{result_df.shape} records found\n")

    result_df.to_csv(f"{output_folder_name}/{cat.replace(' ','_').lower()}.csv", index =False)

    print("csv SAVED")


scrapping data for 2025 Boys

show 100 records per page

scrolling through all the pages....

(473, 8) records found

csv SAVED
scrapping data for 2024 Boys

show 100 records per page

scrolling through all the pages....

(1745, 7) records found

csv SAVED
scrapping data for 2023 Boys

show 100 records per page

scrolling through all the pages....

(2822, 12) records found

csv SAVED
scrapping data for 2025 Girls

show 100 records per page

scrolling through all the pages....

(297, 9) records found

csv SAVED
scrapping data for 2024 Girls

show 100 records per page

scrolling through all the pages....

(1042, 7) records found

csv SAVED
scrapping data for 2023 Girls

show 100 records per page

scrolling through all the pages....

(1003, 7) records found

csv SAVED


In [8]:
print("completed")
driver.close()

completed
